In [1]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

loader = DirectoryLoader("C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload", glob="*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()
print('.....document_loaded.....')

.....document_loaded.....


In [5]:
AZURE_OPENAI_ENDPOINT="https://azureopenai16.openai.azure.com"
AZURE_OPENAI_API_KEY="75db73a3b9da40b0b6e0e98273a6029f"
COHERE_API_KEY="BWR8YyveaadqsWa8Ty0FM0vEIAysgJgnjhZVkRP1"

In [6]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings
import chromadb

storage_path = "../vectordb"

chroma_client = chromadb.PersistentClient(storage_path)
collection = "test"
chroma_vector_search_index_name = "vector_index"
EMBEDDING_MODEL = "ada0021_6"  

#embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_type="azure",
    openai_api_version="2024-02-01",
    openai_api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    model=EMBEDDING_MODEL,
    allowed_special={'<|endoftext|>'}
)

In [7]:
chunk_size = 1000
chunk_overlap = int(0.15*chunk_size)

In [8]:
from rag.chunking import fixed_token_split, recursive_split

splits = fixed_token_split(docs, chunk_size, chunk_overlap)
print("split complete")

USER_AGENT environment variable not set, consider setting it to identify your requests.


split complete


In [9]:
document = splits[10]
print(document.page_content)

List of Tables
1.1 Summary of the UPDRS Item 3.4 (Finger Tapping) rating scale (Goetz et al. 2008) 2
3.1 Comparing ResNet and Darknet Variants (Redmon & Farhadi 2018) . . . . . . 39
4.1 Hand labelling protocol. Each key point is visualised in Figure 4.4. . . . . . . . 47
4.2 Bounding box ground truth annotation format . . . . . . . . . . . . . . . . . . 53
4.3 Comparison of object detection models used in this study . . . . . . . . . . . . 61
4.4 EfﬁcientNet-B0 Network Architecture with custom top for regression. . . . . . 64
4.5 EfﬁcientNet-B4 Network Architecture with custom top for regression. . . . . . 65
v


In [10]:
from rag.ingestion import create_vector_store
from langchain_openai.chat_models import AzureChatOpenAI

vector_store = create_vector_store(splits)

C:\Users\Hammer\Anaconda3\envs\llama\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\Hammer\Anaconda3\envs\llama\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Added 100 chunks to chroma db


In [11]:
query = "What is  Parkinson’s Disease?"

In [12]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [22]:
#from langchain_community.embeddings import SentenceTransformerEmbeddings
#from langchain_chroma import Chroma

# Prepare the database
#db = Chroma(persist_directory=storage_path, embedding_function=azure_embeddings)
#retrieve = db.as_retriever()
#retrieve.get_relevant_documents(query)

In [ ]:
from rag.retriever import contextualCompressionRetriever

retriever = contextualCompressionRetriever(storage_path, azure_embeddings, k=3)
retriever.get_relevant_documents(query)

In [16]:
vector_db = Chroma(
  persist_directory="/vectordb",
  embedding_function=azure_embeddings
)
 
retriever = vector_db.as_retriever(search_kwargs={"k" : 3})

In [17]:
retriever.get_relevant_documents(query)

[]

In [14]:
from langchain.prompts import ChatPromptTemplate

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)
print(prompt)

Human: 
Answer the question based only on the following context:



---

Answer the question based on the above context: Who is bill Palmer?

